In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

1. First we are importing all the necessary libraries¶


In [ ]:
import seaborn as sns
%matplotlib inline
sns.set(color_codes = True)
from scipy.stats import ttest_1samp, ttest_ind, chi2_contingency
from statsmodels.stats.power import ttest_power
from statsmodels.formula.api import ols      
from statsmodels.stats.anova import _get_covariance,anova_lm

2. Reading the data as a dataframe and stroring it into insurance_dataframe.

In [ ]:
insurance_dataframe = pd.read_csv("../input/insurance-premium-prediction/insurance.csv")
insurance_dataframe.head()

Printing the shape of the data

In [ ]:
insurance_dataframe.shape

Conclusion: The data is having 1338 records and each records have 7 attributes/columns

Printing the datatype of each attribute in the dataframe

In [ ]:
insurance_dataframe.dtypes

Checking the presence of null values

In [ ]:
insurance_dataframe.info()


Conclusion: There is no null values in the dataframe as indicated by the info function
We have another function to check directly the presence of any null values in the dataframe.

In [ ]:
insurance_dataframe.isnull().values.any()

Conclusion: Since the function returned false, we are sure that the dataframe does not contain any null value
We will print the 5-point summary of the dataframe. We have transposed it for better readability

In [ ]:
insurance_dataframe.describe().T

Distribution of BMI, Age, Charges

In [ ]:
sns.distplot(insurance_dataframe['bmi']) #Distribution of BMI

In [ ]:
sns.distplot(insurance_dataframe['age']) #Distribution of Age

In [ ]:
sns.distplot(insurance_dataframe['expenses']) #Distribution of Charges

Checking the presence of outliers in BMI, Age and Charges
For this we would respective plot boxplots, which will show us the outliers, if any

In [ ]:
sns.boxplot(insurance_dataframe['bmi']) 

In [ ]:
sns.boxplot(insurance_dataframe['age'])     

In [ ]:
sns.boxplot(insurance_dataframe['expenses'])

onclusion: The box plot reveals that BMI and Charges have outliers while Age do not have any outliers
Distribution of categorical columns (including children

In [ ]:
sns.swarmplot(insurance_dataframe['sex']) #Distribution of sex

In [ ]:
sns.swarmplot(insurance_dataframe['children']) #Distribution of children

In [ ]:
sns.swarmplot(insurance_dataframe['smoker']) #Distribution of smoker

In [ ]:
sns.swarmplot(insurance_dataframe['region']) #Distribution of region

Also we can make a crosstab of the categorical variable and plot their frequency (count)

In [ ]:
#Distribution of sex
sex_plot = pd.crosstab(index = insurance_dataframe["sex"], columns="count")     
sex_plot.plot.bar()

In [ ]:
#Distribution of children
children_plot = pd.crosstab(index = insurance_dataframe["children"], columns="count")     
children_plot.plot.bar()

In [ ]:
#Distribution of smoker
smoker_plot = pd.crosstab(index = insurance_dataframe["smoker"], columns="count")     
smoker_plot.plot.bar()

In [ ]:
#Distribution of region
region_plot = pd.crosstab(index = insurance_dataframe["region"], columns="count")     
region_plot.plot.bar()

Pairplot of all columns of dataframe

In [ ]:
sns.pairplot(insurance_dataframe, hue='sex')

In [ ]:
sns.pairplot(insurance_dataframe, hue='smoker')

In [ ]:
sns.pairplot(insurance_dataframe, hue='region')

Do charges of people who smoke differ significantly from the people who don't?
Approach: We would apply 2 sample Ttest
Null Hypothesis Ho = There is no relation between smokers and charges
Alternate Hypothesis H1 = There is a relation between smokers and charges

In [ ]:
smoker_dataframe = insurance_dataframe[insurance_dataframe['smoker']=='yes'] #Creating a sub-dataframe for smokers
smoker_dataframe.head() 

In [ ]:
nonsmoker_dataframe = insurance_dataframe[insurance_dataframe['smoker']=='no'] #Creating a sub-dataframe for non-smokers
nonsmoker_dataframe.head()

In [ ]:
# finding the ttest independence between the charges of smokers and non smokers
t_statistic, p_value = ttest_ind(smoker_dataframe['expenses'], nonsmoker_dataframe['expenses'])
print(t_statistic, p_value)

In [ ]:
# When P value < 0.05 reject the null hypothesis, otherwise accept
if p_value < 0.05:
    print("Reject H0, the charges of smokers differ significantly from the charges of non smokers.")
else:
    print("Accept H0, the charges of smokers do not differ significantly from the charges of non smokers.")

Conclusion: Since the P-Value is less than the accepted standard of 0.05, we reject the null hypothesis.
Hence There is a relation between smokers and charges
Computing the power of the test

In [ ]:
(smoker_dataframe['expenses'].mean()-nonsmoker_dataframe['expenses'].mean())/np.sqrt(((smoker_dataframe['expenses'].size-1)*np.var(smoker_dataframe['expenses']) + (nonsmoker_dataframe['expenses'].size-1)*np.var(nonsmoker_dataframe['expenses']))/(smoker_dataframe['expenses'].size + nonsmoker_dataframe['expenses'].size-2))

In [ ]:
print(ttest_power(3.165, nobs = 1338, alpha = 0.05, alternative = 'two-sided'))


Conclusion: Power of the test is 100%
Does bmi of males differ significantly from that of females?
Approach: We would apply 2 sample Ttest
Null Hypothesis Ho = There is no relation between bmi and gender
Alternate Hypothesis H1 = These is a relation between bmi and gender

In [ ]:
male_bmi_dataframe = insurance_dataframe[insurance_dataframe['sex']=='male'] #Creating a sub-dataframe for all males
male_bmi_dataframe.head()

In [ ]:
female_bmi_dataframe = insurance_dataframe[insurance_dataframe['sex']=='female'] #Creating a sub-dataframe for all females
female_bmi_dataframe.head()

In [ ]:
# finding the ttest independence between the bmi of male and female
t_statistic, p_value = ttest_ind(male_bmi_dataframe['bmi'], female_bmi_dataframe['bmi'])
print(t_statistic, p_value)

In [ ]:
# When P value < 0.05 reject the null hypothesis, otherwise accept
if p_value < 0.05:
    print("Reject H0, the bmi of males differ significantly from bmi of females")
else:
    print("Accept Ho, the bmi of males does not differ significantly from bmi of females")

Conclusion: Since the P-Value is more than the accepted standard of 0.05, we fail to reject the null hypothesis.
Hence There is no relation between bmi and gender
Computing the power of the test

In [ ]:
(male_bmi_dataframe['bmi'].mean()-female_bmi_dataframe['bmi'].mean())/np.sqrt(((male_bmi_dataframe['bmi'].size-1)*np.var(male_bmi_dataframe['bmi']) + (female_bmi_dataframe['bmi'].size-1)*np.var(female_bmi_dataframe['bmi']))/(male_bmi_dataframe['bmi'].size + female_bmi_dataframe['bmi'].size-2))

In [ ]:
print(ttest_power(0.093, nobs = 1338, alpha = 0.05, alternative = 'two-sided'))

Conclusion: Power of the test is 92.65%
Is the proportion of smokers significantly different in different genders?
Approach: We would apply Chi-square test as we need to find out the relation between 2 categorical variables
Null Hypothesis H0: Proportion of smokers is independent of genders
Alternate Hypothesis H1: Proportion of smokers is not independent of genders

In [ ]:
# Preparing the crosstab function
cont = pd.crosstab(insurance_dataframe['sex'],insurance_dataframe['smoker'])

In [ ]:
cont #Printing the contingency table

In [ ]:
# Applying Chi-square contigency to find the P value
chi2_contingency(cont)

In [ ]:
p_value = 0.006548143503580696 #From previous output


# When P value < 0.05 reject the null hypothesis, otherwise accept
if p_value < 0.05:
    print("Reject H0, the proportion of smokers is not independent on genders")
else:
    print("Accept Ho, the proportion of smokers is independent on genders")

Conclusion: Since the P-Value is less than the accepted standard of 0.05, we reject the null hypothesis.
Hence,The proportion of smokers is not independent on genders
Is the distribution of bmi across women with no children, one child and two children, the same ?
Approach: We would apply ANOVA
Null Hypothesis H0 : BMI has no relation with women and number of childen
Alternate Hypothesis H1: BMI has a relation with women and number of childen

In [ ]:
#We are construction a new dataframe with conditions where sex is female and number of children 0,1 or 2
distribution_dataframe = insurance_dataframe[(insurance_dataframe['sex']=='female') & (insurance_dataframe['children'] < 3)]
distribution_dataframe.head(20)

In [ ]:
distribution_dataframe.info()

In [ ]:
# Converting children into categorical variables
distribution_dataframe.children = pd.Categorical(distribution_dataframe.children)

In [ ]:
# Converting sex and children into categorical variables
distribution_dataframe.sex = pd.Categorical(distribution_dataframe.sex)

In [ ]:
distribution_dataframe.info()

In [ ]:
# Applying anova function where BMI is a function of children
formula = 'bmi ~ C(children)'
model = ols(formula, distribution_dataframe).fit()
aov_table = anova_lm(model)
print(aov_table)

In [ ]:
p_value = 0.715858 #From the above output

# When P value < 0.05 reject the null hypothesis, otherwise accept
if p_value < 0.05:
    print("Reject H0, the means of different number of children are significantly different")
else:
    print("Accept Ho, the means of different number of children are equal")

Conclusion: Since the P-Value is more than the accepted standard of 0.05, we fail to reject the null hypothesis.
Hence BMI has no relation with women and number of childen